# 🎓 학생 성과 예측 - 발표용 종합 분석

## 프로젝트 개요

**논문**: Predicting Student Performance in Higher Education Institutions Using Decision Tree Analysis

**목표**: 의사결정 트리를 사용하여 학생의 학업 성공/실패를 예측하고, 영향 요인을 분석

---

In [ ]:
# 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image, display, Markdown
import sys
sys.path.append('../src')

# 한글 폰트 설정 (선택사항)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# 스타일 설정
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

%matplotlib inline

print("✓ 라이브러리 로드 완료")

---

# Part 1: 데이터 개요

## 1.1 데이터셋 정보

In [ ]:
# 데이터 로드
df = pd.read_csv('../data/student_data_processed.csv')
df_numeric = pd.read_csv('../data/student_data_processed_numeric.csv')

print("=" * 80)
print("데이터셋 기본 정보")
print("=" * 80)
print(f"\n총 학생 수: {len(df):,}명")
print(f"총 특성 수: {len(df.columns) - 1}개")
print(f"\n클래스 분포:")
print(df['Failed'].value_counts())
print(f"\n비율:")
print(df['Failed'].value_counts(normalize=True).apply(lambda x: f"{x:.1%}"))

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 파이 차트
ax = axes[0]
df['Failed'].value_counts().plot(kind='pie', ax=ax, autopct='%1.1f%%',
                                 colors=['#2ecc71', '#e74c3c'],
                                 labels=['Passed', 'Failed'])
ax.set_title('학생 성과 분포', fontsize=14, fontweight='bold')
ax.set_ylabel('')

# 바 차트
ax = axes[1]
counts = df['Failed'].value_counts()
bars = ax.bar(['Passed', 'Failed'], counts, color=['#2ecc71', '#e74c3c'], alpha=0.7)
ax.set_title('학생 수 비교', fontsize=14, fontweight='bold')
ax.set_ylabel('학생 수')

# 값 표시
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
           f'{int(height)}',
           ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

## 1.2 질문 카테고리

In [ ]:
categories = {
    '인구통계': 'Q1-Q5 (나이, 성별, 학년, 학과 등)',
    '사회적 정보': 'Q6-Q11 (결혼 여부, 직업, 부모 정보 등)',
    '학업 정보': 'Q12-Q17 (낙제 과목, 결석, 학점, GPA 등)',
    '학습 기술': 'Q18-Q21 (노트 작성, 시간 계획 등)',
    '동기부여': 'Q22-Q26 (학업 동기, 목표 등)',
    '대인관계': 'Q27-Q30 (친구 관계, 감정 조절 등)',
    '건강': 'Q31-Q34 (운동, 정서적 건강 등)',
    '시간 관리': 'Q35-Q39 (일정 계획, 우선순위 등)',
    '재정 관리': 'Q40-Q44 (돈 관리, 저축 등)',
    '개인 목표': 'Q45-Q49 (교육 목표, 진로 계획 등)',
    '진로 계획': 'Q50-Q53 (직업 정보, 필요 역량 등)',
    '자원': 'Q54-Q56 (재정 자원, 지원 등)',
    '자존감': 'Q57-Q60 (자신감, 능력에 대한 믿음 등)'
}

print("\n질문 카테고리 분류:")
print("=" * 80)
for idx, (category, questions) in enumerate(categories.items(), 1):
    print(f"{idx:2d}. {category:15s}: {questions}")

---

# Part 2: 탐색적 데이터 분석 (EDA)

## 2.1 인구통계 분석

In [ ]:
# EDA 이미지 표시
display(Image(filename='../results/eda_demographics.png'))

### 인사이트:
- 성별, 나이, 학과 등 인구통계적 요인은 성과에 큰 차이를 보이지 않음
- 일하는 학생들의 성과가 약간 낮은 경향

## 2.2 학업 요인 분석

In [ ]:
display(Image(filename='../results/eda_academic.png'))

### 인사이트:
- **낙제 과목 수 (Q12)**: 목표 변수 생성의 기준
- **결석일 (Q13)**: 결석이 많을수록 낙제 비율 증가
- **GPA (Q15)**: 명확한 상관관계 - 높은 GPA = 낮은 낙제율
- **학점 수 (Q14)**: 적절한 학점 수가 중요

## 2.3 카테고리별 평균 점수 비교

In [ ]:
display(Image(filename='../results/eda_likert_categories.png'))

### 인사이트:
- **합격 학생**이 모든 카테고리에서 일관되게 높은 점수
- 특히 차이가 큰 카테고리:
  - 동기부여 (Motivation)
  - 학습 기술 (Study Skills)
  - 시간 관리 (Time Management)
  - 자존감 (Self-Esteem)

## 2.4 통계적 유의성 분석

In [ ]:
# 통계 테스트 결과 로드
stats_df = pd.read_csv('../results/statistical_tests.csv')

print("통계적으로 유의미한 차이를 보이는 질문 (p < 0.05):")
print("=" * 80)

significant = stats_df[stats_df['Significant'] == 'Yes']
print(f"\n총 {len(significant)}개 질문이 유의미한 차이를 보임 (전체 {len(stats_df)}개 중)\n")

print("\nTop 10 가장 큰 차이를 보이는 질문:")
print("-" * 80)
top_10 = stats_df.head(10)
for idx, row in top_10.iterrows():
    print(f"\n{idx+1}. {row['Question']}")
    print(f"   합격 평균: {row['Passed_Mean']:.3f}")
    print(f"   불합격 평균: {row['Failed_Mean']:.3f}")
    print(f"   차이: {row['Mean_Diff']:.3f}")
    print(f"   p-value: {row['P_Value']:.4f}")
    print(f"   Cohen's d: {row['Cohens_D']:.3f}")

# 시각화
display(Image(filename='../results/eda_statistical_differences.png'))

---

# Part 3: 모델 성능 분석

## 3.1 모델 비교

In [ ]:
# 모델 성능 로드
results_full = pd.read_csv('../results/model_comparison_full.csv')
results_filtered = pd.read_csv('../results/model_comparison_filtered.csv')

print("모델 성능 비교")
print("=" * 80)

print("\n1) 전체 속성 (60개) 사용:")
print("-" * 80)
print(results_full[['model_name', 'accuracy_mean', 'tp_rate_mean', 
                     'fp_rate_mean', 'precision_mean', 'recall_mean']].to_string(index=False))

print("\n\n2) 필터링된 속성 (상위 40개) 사용:")
print("-" * 80)
print(results_filtered[['model_name', 'accuracy_mean', 'tp_rate_mean', 
                         'fp_rate_mean', 'precision_mean', 'recall_mean']].to_string(index=False))

# 시각화
display(Image(filename='../results/figure2_performance_comparison.png'))

### 결론:
- **J48 (C4.5) 알고리즘**이 최고 성능
- 속성 필터링 후 모든 모델의 성능 향상
- 불필요한 특성 제거가 모델 성능 개선에 도움

## 3.2 특성 중요도

In [ ]:
display(Image(filename='../results/j48_feature_importance.png'))

In [ ]:
# 특성 중요도 데이터 로드
importance_df = pd.read_csv('../results/j48_feature_importance.csv')

print("\nTop 10 가장 중요한 특성:")
print("=" * 80)
for idx, row in importance_df.head(10).iterrows():
    print(f"{idx+1:2d}. {row['Feature']:10s}: {row['Importance']:.4f}")

---

# Part 4: Decision Tree 해석

## 4.1 의사결정 트리 시각화

In [ ]:
display(Image(filename='../results/figure3_j48_tree.png'))

## 4.2 의사결정 규칙 해석

In [ ]:
# 의사결정 규칙 로드
try:
    rules_df = pd.read_csv('../results/J48_decision_rules.csv')
    
    print("주요 의사결정 규칙 (상위 10개):")
    print("=" * 80)
    
    for idx, row in rules_df.head(10).iterrows():
        print(f"\n규칙 {idx+1}:")
        print(f"  조건: {row['path']}")
        print(f"  결과: {row['prediction']}")
        print(f"  신뢰도: {row['confidence']:.1%}")
        print(f"  샘플 수: {row['samples']}")
except FileNotFoundError:
    print("의사결정 규칙 파일을 찾을 수 없습니다.")
    print("tree_interpretation.py를 먼저 실행해주세요.")

## 4.3 특성 사용 빈도

In [ ]:
try:
    display(Image(filename='../results/J48_feature_usage.png'))
    
    usage_df = pd.read_csv('../results/J48_feature_usage.csv')
    print("\n의사결정 트리에서 가장 자주 사용되는 특성:")
    print("=" * 80)
    print(usage_df.head(15).to_string(index=False))
except FileNotFoundError:
    print("특성 사용 빈도 파일을 찾을 수 없습니다.")

---

# Part 5: 핵심 인사이트 및 제언

## 5.1 주요 발견사항

In [ ]:
insights = """
# 핵심 인사이트

## 1. 학업 성공의 주요 예측 요인

### ✅ 높은 영향력 요인:
1. **학업 관련 지표**
   - GPA (학점)
   - 수강 학점 수
   - 결석 일수

2. **심리적 요인**
   - 학업에 대한 동기부여
   - 자기 효능감 (자신감)
   - 수업에 대한 흥미

3. **행동적 요인**
   - 시간 관리 능력
   - 학습 기술
   - 계획 및 조직화 능력

### ❌ 낮은 영향력 요인:
- 나이, 성별
- 결혼 여부
- 부모의 직업
- 거주지

## 2. 주요 발견

💡 **학생의 성공은 타고난 환경보다 본인의 태도와 행동에 더 큰 영향을 받는다**

## 3. 모델 성능

- J48 (C4.5) 알고리즘이 최고 성능
- 속성 선택을 통해 모델 성능 향상
- 높은 예측 정확도로 조기 개입 가능
"""

display(Markdown(insights))

## 5.2 실용적 제언

In [ ]:
recommendations = """
# 실용적 제언

## 📚 학생들을 위한 제언

### 1. 동기부여 유지
- 명확한 학업 목표 설정
- 작은 성취를 축하하기
- 학습의 의미 찾기

### 2. 효과적인 학습 전략
- 체계적인 노트 작성
- 규칙적인 복습 계획
- 능동적 학습 방법 활용

### 3. 시간 관리
- 주간 학습 계획 수립
- 우선순위 설정
- 미루기 습관 극복

### 4. 자기 효능감 향상
- 긍정적 자기 대화
- 도전적 목표 설정
- 성공 경험 축적

---

## 👨‍🏫 교수자를 위한 제언

### 1. 조기 경보 시스템
- 학기 초 위험 학생 식별
- 정기적 모니터링
- 데이터 기반 개입

### 2. 맞춤형 지원
- 개별 학생 특성 파악
- 차별화된 지원 전략
- 멘토링 프로그램

### 3. 동기부여 강화
- 흥미로운 수업 설계
- 실생활 연결
- 긍정적 피드백

### 4. 학습 기술 교육
- 효과적 학습 전략 지도
- 시간 관리 워크숍
- 스터디 그룹 활성화

---

## 🏛️ 기관/행정가를 위한 제언

### 1. 시스템 구축
- AI 기반 학생 성과 예측 시스템 도입
- 통합 학생 지원 플랫폼
- 데이터 기반 의사결정 문화

### 2. 자원 배분
- 효과 입증된 프로그램에 집중 투자
- 학습 지원 센터 강화
- 상담 서비스 확대

### 3. 정책 개발
- 출석 관리 정책
- 학업 부진 학생 지원 정책
- 조기 개입 프로토콜
"""

display(Markdown(recommendations))

---

# Part 6: 결론

## 6.1 연구 요약

In [ ]:
conclusion = """
# 🎯 결론

## 연구 목표 달성

✅ **목표 1**: 학생 성과 예측 모델 구축
- J48 의사결정 트리로 높은 정확도 달성
- 3가지 알고리즘 비교 및 최적 모델 선정

✅ **목표 2**: 영향 요인 분석
- 60개 특성 중 핵심 요인 식별
- 통계적 검증을 통한 유의미한 차이 확인

✅ **목표 3**: 실용적 인사이트 도출
- 학생, 교수자, 기관을 위한 구체적 제언
- 조기 개입 시스템 설계 기반 마련

---

## 핵심 메시지

### 💎 "데이터와 AI를 활용하여 모든 학생의 성공을 지원하자"

1. **예측**: AI 모델로 위험 학생 조기 발견
2. **이해**: 데이터 분석으로 핵심 요인 파악
3. **개입**: 증거 기반 맞춤형 지원
4. **개선**: 지속적 모니터링과 피드백

---

## 한계점 및 향후 연구

### 한계점
- 합성 데이터 사용 (실제 데이터 수집 필요)
- 단일 기관 데이터 (일반화 검증 필요)
- 정적 분석 (시계열 분석 부족)

### 향후 연구 방향
- 실제 학생 데이터로 검증
- 종단 연구 (longitudinal study)
- 개입 효과 측정
- 다양한 기관 간 비교
- 딥러닝 모델 적용

---

## 🙏 감사합니다!

**질문이 있으신가요?**
"""

display(Markdown(conclusion))

---

# Appendix: 추가 분석 실행

## A.1 고급 EDA 실행

In [ ]:
# 고급 EDA 실행 (필요시)
# %run ../src/advanced_eda.py

## A.2 Decision Tree 해석 실행

In [ ]:
# Tree 해석 실행 (필요시)
# %run ../src/tree_interpretation.py